In [ ]:
from gapml.utils.img_tools import img_utils

### Type Of FolderTrees

#### tree = 1
```
[root_path] "folder_name"/..  
    [subfolder] class_0/..  
    [subfolder] class_1/..  
    [subfolder] errors/..
```

#### tree = 2
```
[root_path] "folder_name"/..  
    [subfolder] train_tr/..  
        [subfolder] class_0/..  
        [subfolder] class_1/..  
    [subfolder] train_val/..  
        [subfolder] class_0/..  
        [subfolder] class_1/..  
    [subfolder] test/test/..  
    [subfolder] errors/..  
```

In [ ]:
#initialize
gap = img_utils(root_path='flower_photos')

In [ ]:
%pwd

In [ ]:
%ls

In [ ]:
%ls flower_photos/

In [ ]:
# 1to2
gap.transform(shufle=True, img_split=0.2)

In [ ]:
%ls

In [ ]:
%ls flower_photos_t2/

In [ ]:
gap = img_utils(root_path='flower_photos_t2/train_tr', transf='2to1')

In [ ]:
gap.transform()

In [ ]:
%ls

In [ ]:
%ls flower_photos/

In [ ]:
gap = img_utils(root_path='flower_photos')

In [ ]:
gap.img_container(action='copy', spl=20)

In [ ]:
%ls

In [ ]:
%ls flower_photos_spl/

In [ ]:
gap = img_utils(root_path='flower_photos', tree=2)

In [ ]:
gap.img_container(action='copy', spl=15, shufle=True, img_split=0.2)

In [ ]:
%ls flower_photos_t2_spl/

In [ ]:
gap = img_utils(root_path='flower_photos_spl')

In [ ]:
%ls flower_photos/roses/

In [ ]:
gap.img_rename()# text=True text='test'

In [ ]:
%ls flower_photos/roses/

In [ ]:
gap.img_replace(old='st', new='sted', img_id=False)

In [ ]:
%ls flower_photos/roses/

In [ ]:
gap = img_utils(root_path='flower_photos_t2_spl/train_tr', tree=2)

In [ ]:
%ls flower_photos_t2_spl/train_tr/roses/

In [ ]:
gap.img_rename()# text=True text='test'

In [ ]:
%ls flower_photos_t2_spl/train_tr/roses/

In [ ]:
gap.img_replace(old='st', new='sted', img_id=True)

In [ ]:
%ls flower_photos_t2_spl/train_tr/roses/

In [ ]:
gap = img_utils(root_path='flower_photos_spl', rm=True)

In [ ]:
gap = img_utils(root_path='flower_photos_t2_spl', rm=True)

In [ ]:
%ls